# Dynamic Systems

All physical systems that store and transform energy are all dynamic systems that cannot react instantaneously
to an external solicitation. This because if an energy transformation were to be instantaneous, it would require
an infinite power ($P=dW/dt$), which is impossible to deliver.

These systems are characterized by their capacity to act or not in a stable way, in a fast way, in an oscillating
way, ...

When the underlying physics of dynamic systems is sufficiently known, we have mathematical models capable
of describing their temporal behavior: differential equations. Matlab allows the integration of linear differential
equations with constant coefficients (LTI: Linear Time Invariant), expressed in different forms (and thus their
numerical solutions), but also of non-linear differential equations (for which the most appropriate simulation tool
is Simulink but it will not be discussed here).

As an illustration, let's take the following example: if a force applied to a mass is at the origin of an acceleration,
it is by integrating this force (divided by the mass) that we will obtain the velocity as a function of time and, by
integrating the velocity, we will obtain the distance traveled. So we express the derivatives, Matlab integrates
them. That's easy!

We will continue this tutorial with this well known example: the system mass ($M$), spring ($K$), damper ($R_f$), of
position $y$, subjected to an external force $u$.

![](./1_image1.png)

with $M = 0.5 kg$; $R_f=0.2kg/s$ and $K=1kg/s^2$

The first step in the analysis of a physical system is to write down the laws of physics that govern its operation.

Here, the dynamic equilibrium equation of the forces ($\sum_i F_i = M \cdot a$) on the mass $M$ is writen : $M \frac{d^2y}{dt} + R_f \frac{dy}{dt} + Ky = u$ or in a normalized way : $\frac{d^2y}{dt^2} + \frac{R_f}{M}\frac{dy}{dt} + \frac{K}{M}y = \frac{1}{M}u$

Recall that this differential equation can, **only because it is an LTI equation**, be transformed into a polynomial expression as a function of the variable "$s$" by passing in the Laplace domain.

This passage in the Laplace domain is done with the Laplace transform, and allows us :

- to make our life much easier : we will manipulate algebraic equations rather than differential equations
- to quickly judge the dynamics and stability of the system by identifying its poles (the roots of the
denominator)
- to obtain directly the harmonic response of the system, i.e. in sinusoidal regime after the transient phase,
by replacing "$s$" by "$j\omega$".

Our differential equation becomes, by the Laplace transform : $s^2Y(s) + \frac{R_f}{M}sY(s) + \frac{K}{M}Y(s) = \frac{1}{M}U(s)$

The transfer function is the ratio of the output y to the input u : $$H(s) = \frac{Y(s)}{U(s)} = \frac{\frac{1}{M}}{s^2 + \frac{R_f}{M}s + \frac{K}{M}}$$

This transfer function $H(s)$ will serve as an illustrative example throughout this tutorial. As soon as we talk about the $H(s)$ system, it will be good to remember that the physics behind this transfer function is this mass-spring-damper system.

*NB: If the equation relating to this system were a non-linear differential equation, it would be necessary to work with the tools that are briefly presented at the end of this tutorial, or else it would be necessary to accept to work with small variations of the quantities around an operating point of this system to be able to apply the Laplace transform. This amounts to linearizing this system and allows to use all the theory related to transfer functions. This linearization approach is the one most frequently encountered in industry. Let's remember that the most frequent objective in industry is to control a system with a controller. And it turns out that the LTI tools allowing the implementation of this control are much more understandable and simple to implement than the non linear techniques, which explains why these solutions are preferred.*

## Transfer functions creation (continuous or discrete) in Python

Python doesn't know about the $s$ variable out of the box. We need to use third party packages to work with transfer functions. In this lab, we will use two of these package: `sympy` and `control`. `sympy` is a general symbolic math package which will pretty much let us use the theoretical formulas as is. The `control` package is specialized for transfert function representation and manipulation so it will be very useful here but we'll see that it can't do everything.

We will first begin with the transfert function creation with the `sympy` package. We begin by creating the $s$ symbol:

In [ ]:
import sympy as sp

s_sp = sp.symbols('s')
s_sp

This variable (Laplace variable) can then be manipulated to write the polynomials of the transfer functions:

In [ ]:
Hsp = 2/(s_sp**2+0.4*s_sp+2)
Hsp # to make jupyter display the transfer function

With the `control` package, the "$s$" variable creation is done with the instruction `tf`:

In [ ]:
import control as ct

s=ct.tf('s')

which let us write the same transfert function:

In [ ]:
assert s is not None  # to please the type checker
H = 2/(s**2+0.4*s+2)

print(H)

Note that the instruction `tf` can be used in another way, without creating the variable $s$, by giving it the coefficients of the polynomials in $s$: `ct.tf(num,den)` where `num` is the polynomial of the coefficients in $s$ of the numerator and `den` that of the denominator.

It is also possible to create a transfer function in Python by providing the zeros, the poles and the gain, via the instruction `ct.zpk`.

These ways of doing things being less intuitive, we will prefer the method based on the creation of the variable $s$, and refer you to the `control` package help for the rest.

In the same way, we can create a transfer function not in continuous but in discrete, by creating beforehand the variable $z$, as well as the sampling time `Ts` of application. Unlike Matlab, in Python you must use the `ct.tf(num, den, Ts)` form to create a discrete-time transfer function. To create the $z$ variable, choose the `num` and `den` so that the transfer function simplifies to $z$:

$$z = \frac{1z + 0}{1}$$

In [ ]:
Ts = 0.1 # this is the sampling time, expressed in seconds
z = ct.tf([1, 0], [1], Ts) 
print(z)

In [ ]:
assert z is not None # to please the type checker
Hz=2/(z**2+0.4*z+2)  # Discrete transfer function, with the sample time
                     # (Ts) = 0.1s, as the "z" variable was created specifying
                     # this value
print(Hz)

**Pay attention**, it is important to note that the transfer function `Hz` created above is not the equivalent of the transfer function `H` in discrete, even if the numerator and denominator are identical!

To discretize the system `H` and obtain the discrete version, we can use the "continue to discrete" command, specifying the continuous system to be discretized and the sampling period Ts : `Sysd = ct.c2d(Sysc,Ts)`.

By default, the discretization method used is that of the zero-order blocker:

In [ ]:
Hz = ct.c2d(H, Ts) # We get a discrete transfert function (using the z variable).
                   # The polynomials (numerator and denominator) are different from
                   # the ones we had in continuous H.
print(Hz)

We can see here that the global order of `Hz` is $z^{-1}$, which expresses well that the influence of the input on the output will be perceived only after a sampling time.

## Finding and plotting a transfer function poles and zeros

As already mentioned, the poles of this transfer function correspond to the roots of the denominator (which can be real or complex conjugates), and characterize the dynamics of the system and its stability. Indeed, when we use the inverse Laplace transform to return to the time domain, the answer we obtain will be a sum of exponentials defined by the position of the poles in the Laplace domain:

- The real part of the poles will define the speed of decay or growth of the exponential : $e^{t \cdot p_r}$ with $p_r$ being the real part of the pole : One can understand that poles with a positive real part will give rise to increasing exponentials, and thus to a system which will not converge and will be unstable. **A system will be stable if all its poles are located to the left of the imaginary axis in the complex plane**. One can also observe that $e^{t \cdot p_r}$ can be classically written $e^{\frac{t}{\tau}}$ when dealing with differential equations, and that therefore the real part of a pole is the inverse of its time constant : the further the pole is from the imaginary
axis, the faster the associated time constant will be, and the less its influence on the transient will be important. Conversely, a pole close to the imaginary axis will be considered as a dominant pole.
- The imaginary part of the poles will lead in the time domain to a complex exponential, or a sinusoid. This implies that **if there are complex poles conjugated in the complex plane, it will lead us to have an oscillating response in the time domain**. The imaginary part of the poles defines the pulsation of this sine. And the combination of the real and imaginary parts defines what the overshoot will be when responding to a step.

The poles can be obtained easily, with the roots function of `numpy`:

In [ ]:
import numpy as np

assert isinstance(H, ct.TransferFunction) # to please the type checker
np.roots(H.den[0][0])

or via the 'poles' method of the transfer function object :

In [ ]:
assert isinstance(H, ct.TransferFunction) # to please the type checker
H.poles()   # or ct.poles(H)

Similarly, if the system had zeros, we could find them by looking for the roots of the numerator :

In [ ]:
assert isinstance(H, ct.TransferFunction)
np.roots(H.num[0][0])  # We get an empty vector as the numerator has no zero
                       # in this specific case.

or via the `zeros` method.

In [ ]:
H.zeros()

If you use `sympy`, you can use yhe `sympy` way to get roots:

In [ ]:
zeros = sp.solve(Hsp, s_sp)
poles = sp.solve(s_sp**2+0.4*s_sp+2, s_sp)
print(zeros)
print(poles)

We get sympy complex numbers. If we want regular python complex we can do:

In [ ]:
poles = [complex(p) for p in poles]
poles

As mentioned above, we can create a transfer function from zeros, poles, and gain with the `ct.zpk` function. The gain used for this function **is not the static gain** but is instead the ratio between the coefficients of the highest order terms of the numerator and denominator of the transfer function.

In [ ]:
G_zpk = H.num[0][0][0] / H.den[0][0][0]
G_zpk

In [ ]:
print(ct.zpk(H.zeros(), H.poles(), G_zpk))

The `ct.pzmap` function allows to draw the poles and zeros of a system in the complex plane :

In [ ]:
from matplotlib import pyplot as plt

# better plots in jupyter
%config InlineBackend.figure_formats = ['svg']

ct.pzmap(H, grid=True)
plt.show()


### What if we work with a discrete system ?

We have the same tools to find and draw the poles and to draw. Remember that for a discrete system, the stability condition is that the poles are located in a circle of unit radius centered at $(0,0)$.

In [ ]:
Hz.poles()

In [ ]:
ct.pzmap(Hz, grid=True)
plt.show()

## Handling of LTI dynamic objects

When the output of the `H1` system is connected to the input of the `H2` system, the global transfer function is obtained by the series connection, thus mathematically by a multiplication of the 2 transfer functions.

In the time domain, the convolution integral should have been used instead of this simple product in the Laplace domain.

![](./1_image2.png)

In [ ]:
assert s is not None

H5 = 1 / (5*s + 1)
H6 = 1 / (10*s + 1)
print(H5 * H6)

It works with `sympy` representation as well:

In [ ]:
H5sp = 1 / (5*s_sp + 1)
H6sp = 1 / (10*s_sp + 1)
H5sp * H6sp

`sympy` contains many functions to simplify mathematical expressions (`sp.simplify()`, `sp.expand()`, `sp.factor()`, `sp.collect()`, `sp.apart()`, `sp.cancel()`).

Here `sp.cancel()` is a good fit to get the output we used to:

In [ ]:
sp.cancel(H5sp * H6sp)

The parallel connection is obtained with the " + ":

![](./1_image3.png)

In [ ]:
print(H5 + H6)

Also with `sympy`

In [ ]:
sp.cancel(H5sp + H6sp)

When a feedback loop is closed, the `ct.feedback` function is used to obtain the transfer function of the looped system:

![](./1_image4.png)

In [ ]:
print(ct.feedback(H5, H6))

or without the feedback function :

In [ ]:
print(H5 / (1 + H5 * H6))

This is also the way we will do it with `sympy`:

In [ ]:
H5sp / (1 + H5sp * H6sp)

When manipulating these transfer functions, the `control` package does not simplify the poles and zeros on its own initiative when possible.

In [ ]:
assert s is not None
H2 = (s + 2) / ((s + 3) * (s + 5))
H3 = (s + 3) / (s + 4)
print(H3 * H2)

This often occurs when multiple operations are performed on transfer functions. We can therefore simplify the expression, which is done with the `ct.minreal` function:

In [ ]:
print(ct.minreal(H3*H2))

With `sympy`, we can use the simplify mathematical expressions (`sp.simplify()`, `sp.expand()`, `sp.factor()`, `sp.collect()`, `sp.apart()`, `sp.cancel()`):

In [ ]:
H2sp = (s_sp + 2) / ((s_sp + 3) * (s_sp + 5))
H3sp = (s_sp + 3) / (s_sp + 4)
sp.cancel(H3sp * H2sp)

## Temporal responses to a step, an impulse and any input signal (lsim)

With the `ct.step_response` function, the step response of the system will be calculated from $t=0s$ until the transients are done, regardless of which tool the dynamic object was created with (`ct.zpk`, `ct.tf`, ...)

It is worth noting that the `ct.step_response` function executes a unitary amplitude step at time $t=0s$.

In [ ]:
ct.step_response(H).plot()
plt.grid()
plt.show()

By default, Python defines the length of the time vector itself, so as to show the whole transient. It is also possible to impose the time over which the step response must be calculated and plotted.

In [ ]:
T = np.arange(0, 50, 0.1)
ct.step_response(H, T).plot()
plt.grid()
plt.show()

One can also get the data of the step response in 2 vectors `t` & `y`, instead of seeing the step response beeing plotted :

In [ ]:
t, y = ct.step_response(H)
y

In [ ]:
t

In [ ]:
assert t is not None
assert y is not None

plt.plot(t, y)
plt.grid()
plt.show()

**NB:** You will have noticed that the static gain of the system is 1, whereas the zero() instruction had announced a gain (not static) of 2: this gain was that of the numerator for the transfer function written in the Evans form (unit coefficient for the most important power in $s$ of the denominator).

It is good to remember that the static gain, no matter in which form the system is presented, can always be calculated by replacing $s$ by 0 in the transfer function.

In Python, we can get the static gain with the `ct.dcgain` function :

In [ ]:
ct.dcgain(H)

With `sympy`, the step response is the inverse Laplace transform of the transfer function divided by $s$

In [ ]:
t_sp = sp.symbols('t') # We need the `t` symbol

step_resp = sp.inverse_laplace_transform(Hsp/s_sp, s_sp, t_sp)

sp.plot(step_resp, (t_sp, 0, 35))
plt.show()

### What about the discrete time response ?

The step response of a discrete system can also be plotted using the same step command :

In [ ]:
print(Hz)

In [ ]:
T = np.arange(0, 10, 0.1)
t_Hz, y_Hz = ct.step_response(Hz, T)
# The plot method smooth the graph by default so we use the `plt.step` function to show the discretization
plt.step(t_Hz, y_Hz, where='post', label="Discrete system") 
ct.step_response(H, T).plot('r', label="Continuous system")
plt.legend()
plt.show()

One can see the sampling time (0.1 second) on the step response, but also the similarities between the continuous and discrete systems. Note that the discrete response is late compared to the continuous one, the delay beeing the half of the sampling period. It should also be noted that the values are not known between 2 sampling times. A more correct representation of the actually known information would be obtained by showing
the values at the sampling times only, using the `plt.stem` function:

In [ ]:
plt.stem(t_Hz, y_Hz)
plt.show()

The **impulse response** is obtained with the `ct.impulse_response` function :

In [ ]:
ct.impulse_response(H, T).plot()
plt.grid()
plt.show()

and with the same instruction for a discrete system :

In [ ]:
t_Hz, y_Hz = ct.impulse_response(Hz, T)
plt.step(t_Hz, y_Hz, where='post', label="Discrete system")
plt.grid()
plt.show()

With `sympy`, the impulse response is the inverse Laplace transform of the transfer function

In [ ]:
imp_resp = sp.inverse_laplace_transform(Hsp, s_sp, t_sp)

# To plot a sympy expression we must use `sp.plot` which use matplotlib under the hood
sp.plot(imp_resp, (t_sp, 0, 35))
plt.show()

## Time-related response to any signal

The `control` package of Python integrates functions to obtain the temporal responses to Dirac impulse or unit step solicitations. But how to obtain the response to any solicitation?

We will have to use the `ct.forced_response` function, having previously created the input signal manually. This way of doing things can be used to calculate the response to any signal

In [ ]:
t = np.arange(0, 50, 0.1)

u = 5 * t - 50
u[t < 10] = 0
u[t >= 30] = 5 * 30 - 50

_, y = ct.forced_response(H, t, u)

plt.plot(t, u, label="Input")
plt.plot(t, y, label="Response")
plt.grid()
plt.legend()
plt.show()

To do this with `sympy`, first we need to define the input signal symbolically with `sympy`:

In [ ]:
u_sp = sp.Piecewise(
    (0, t_sp <= 10),
    (5 * t_sp - 50, (t_sp > 10) & (t_sp < 30)),
    (5 * 30 - 50, t_sp >= 30)
)

sp.plot(u_sp, (t_sp, 0, 40))
plt.show()

The transfer function is related to the Laplace transforms of the input and output signals by this relation: $$Y(s) = H(s) \cdot U(s)$$

In [ ]:
Usp = sp.laplace_transform(u_sp, t_sp, s_sp, noconds=True)
Ysp = Hsp * Usp
y_sp = sp.inverse_laplace_transform(Ysp, s_sp, t_sp)

graph = sp.plot(u_sp, (t_sp, 0, 40), show=False, label="Input", legend=True)
graph.extend(sp.plot(y_sp, (t_sp, 0, 40), show=False, label="Response"))
graph.show()

When working with `sympy` expressions, it is sometimes useful to get a `numpy` function version of it: 

In [ ]:
y_np = sp.lambdify(t_sp, y_sp, "numpy")

tm = np.linspace(0, 40, 500)
ym = y_np(tm)

plt.plot(tm, ym)
plt.grid()
plt.show()

### Exercise

We now suggest to solve the next exercise that deals with the creation and manipulation of transfer functions.

[1_1_FT_Manipulation.ipynb](./1_1_FT_Manipulation.ipynb)

### Exercise

We now propose to do an exercise that deals with the identification of a system according to a model (a transfer function) imposed by its step response.

[1_2_idf_FT_VDG.ipynb](./1_2_idf_FT_VDG.ipynb)

## Frequency response: the Bode plot

In addition to the temporal analysis of systems, the harmonic (or frequency) analysis allows to obtain information about the behavior of a system that the temporal analysis can not give directly:

- The frequency analysis of the system in open loop is frequently used to deduce the behavior it will have in closed loop,
- The frequency analysis allows to quantify the stability of a system constituted with a feedback (closed loop),
- Knowing that any signal can be reconstructed as a sum of sines of different frequencies, phase and amplitude, the frequency analysis allows to understand some temporal behaviors.

The principle of frequency analysis of a system is as follows: the system is solicited with a sinusoidal input. Knowing that the system is LTI, we know that we will obtain a sinusoidal signal (oscillating at the same frequency as the input signal) at the output of this same system, but that it may have a different amplitude from the input signal, and that a phase shift may appear between this output signal and the input signal. These 2 phenomena are directly linked to the time constants of the system, which, as a reminder, are an image of its inertia. We will therefore analyze how the amplitude ratio of the signals, called the gain (amplitude of the output
signal compared to that of the input signal), and how the phase shift evolves when we vary the frequency of the input signal while keeping its amplitude constant.

It is important to note that this analysis is done in "steady state": we consider that the sinusoidal input signal of the system is present for a sufficiently long time so that all the transients of the system are over.

This analysis is made easier by the use of transfer functions: indeed, it is enough to replace "$s$" by "$j\omega$", which generates a complex fraction. Then to see how the norm and the phase of this one evolve when $\omega$ varies. Let's do it for our example:

Replacing "$s$" by "$j\omega$" in the transfer function, we get the next complex fraction : $$\frac{2}{(j\omega)^2 + 0.4(j\omega) + 2}$$

In [ ]:
w = np.logspace(-2, 2, 100) # Vector of the frequencies for which I want to know the behavior of the system

c=2/((1j*w)**2+0.4*(1j*w)+2) # Matrix of complex fractions for the corresponding frequencies

m=180*np.angle(c)/np.pi # Phase in degrees
g=20*np.log10(np.abs(c)) # Gain in dB

plt.figure()
plt.subplot(2, 1, 1)
plt.semilogx(w, g)
plt.grid()
plt.subplot(2, 1, 2)
plt.semilogx(w, m)
plt.grid()
plt.show()

We have manually rebuild the Bode diagram, which represents the way the gain and phase of the output signal of a system evolve as a function of the frequency of the input signal.

In Python, you can use the `ct.bode` function to directly get the same result:

In [ ]:
ct.bode(H)
plt.show()

As for the temporal responses, it is possible to impose the frequencies at which we want to plot the Bode diagram:

In [ ]:
ct.bode(H, w)
plt.show()

And also we can use the `ct.frequency_response` function to recover the phase and gain for a given frequency:

In [ ]:
w_spec = 5
mag, phase, omega = ct.frequency_response(H, w_spec)
print(mag) # gain not in dB
print(phase) # phase in radian
print(omega)

In [ ]:
magdB = 20*np.log10(mag[0])
phaseDeg = 180*phase[0]/np.pi

print(f"{magdB=}\n{phaseDeg=}")

This allows us to read the value of the static gain of the system, which is read at the zero frequency :

In [ ]:
static_gain, _, _ = ct.frequency_response(H, 0)
print(static_gain[0])

We can see that the static gain of the system is equal to 1 in our case. Remember that it is because the Bode diagram is drawn for a sinusoidal steady state regime that the fact of imposing a zero frequency can read the static gain.

One of the great interests of the representation of systems in the Bode diagram is that it is possible to simply sum the curves of the different elements composing the direct chain (the open loop) of the complete system. This is made possible by the use of the logarithmic axis in this Bode diagram. Indeed, we remember that $log(ab) = log(a) + log(b)$. This property applies only to the Bode diagram, not to the Nyquist diagram which will be described below.

Thus, the frequency response of a transfer function $G(s) = \frac{5}{s^2+5s+6}$ which can be written as the product of 3 independent transfer functions $5$, $\frac{1}{s+3}$ and $\frac{1}{s+2}$ can be easily plotted by drawing the diagrams corresponding to the 3 simple functions and summing them:

In [ ]:
ct.bode(ct.tf(5, 1), None, ':b', label="$5$")
ct.bode(ct.tf(1, [1, 3]), None, ':r', label=r"$\frac{1}{s+3}$")
ct.bode(ct.tf(1, [1, 2]), None, ':g', label=r"$\frac{1}{s+2}$")
ct.bode(ct.tf(1, [1, 5, 6]), None, ':c', label=r"$\frac{1}{s^2+5s+6}$")
ct.bode(ct.tf(5, [1, 5, 6]), label=r"$\frac{5}{s^2+5s+6}$")
plt.legend()
plt.grid()
plt.show()

Time for an exercise using the Bode plot and illustrating that it provides the harmonic response only in steady state:

[1_3_sine_wave_sollicitation.ipynb](./1_3_sine_wave_sollicitation.ipynb)

Let's start a new exercise: solving an electric circuit in steady state and transient:

[1_4_AC_Electrical_circuit_2.ipynb](./1_4_AC_Electrical_circuit_2.ipynb)

## Nyquist plot

The Nyquist diagram represents exactly the same thing as the Bode diagram, but instead of representing the gain and phase as a function of frequency, it is a polar representation on a single curve.

We can imagine that it is the plot of the end of a rotating vector whose origin is placed in $[0;0]$ and whose amplitude and phase vary with frequency. We specify with an arrow on the curve obtained in which direction to follow a growing $\omega$. It should be noted that the gain is not calculated in dB, unlike the Bode diagram.

In the case of our system, for a zero frequency, the gain is unity and the phase zero. And for an infinite frequency, the gain is zero and the phase is -180°. This can be seen on the following diagram.

In [ ]:
ct.nyquist(H) # diagram of the gain versus the phase (where the frequency does not appear)
plt.show()

Time for a new exercice combining Bode and Nyquist plots :

[1_5_Nyquist.ipynb](./1_5_Nyquist.ipynb)

### Black-Nichols plot

Last representation of the frequency response of a system, the Black-Nichols diagram is drawn in a Cartesian graph with in abcissa the phase of the system, and in ordinate the gain expressed in dB, for an increasing pulse.

In [ ]:
ct.nichols(H)
plt.grid()
plt.show()

**Example: Study of the current transient of a non-ideal choke connected to the network, at a phi phase of the sinusoid**

In [ ]:
f = 50             # Network frequency
w = 2 * np.pi * f
L = 0.005          # Self [F]
R = 0.05           # Resistance [R]
Zself = R + 1j*w*L # Impedance for w = 2*pi*f
Yself = 1 / Zself
U = 240            # Voltage
I = U * Yself
I                  #Current ==> Almost only reactive.

In [ ]:
abs(I)   # Its a complex number, let's compute its norm

This value could have been obtained by creating the transfer function corresponding to this system, and looking at the harmonic response for the desired pulse:

In [ ]:
G = ct.tf(1,[L, R])
print(G)

In [ ]:
mag, phase, _ = ct.frequency_response(G, 100*np.pi)
print(mag)
print(phase*180/np.pi)

The current amplitude is therefore the gain multiplied by the voltage amplitude:

In [ ]:
U * mag

This current is correct but only in the steady state, i.e. after the inrush current. As mentioned earlier, the harmonic analysis via Bode, Nyquist, etc. tools only concerns the steady state.

However, the question arises as to how long it takes to reach the steady state?

Let's study the Inrush current for a phase $\pi$ of the voltage at the onset by means of the transfer function, but in time then, by means of the `ct.forced_response` function (it is not possible to do otherwise this time)

In [ ]:
t = np.linspace(0, 0.3, 400)
phi = np.pi     # the phase of the voltage at the time of switching on

u=240*np.sqrt(2)*np.sin(100*np.pi*t+phi) # the instantaneous network voltage

s = ct.tf('s')
Yself = 1 / (R + s*L)
_, a = ct.forced_response(Yself, t, u)     # a = the current = u*Y

plt.figure()
plt.plot(t, u)
plt.plot(t, a, 'r')
plt.show()

© JM Rousseau - D Rouchard - Q Lurkin